<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 13:44:19
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.39 C
-------------------
Today PnL: -58.87 K (-0.42%)
Current PnL: -20.47 L (-13.77%)
CY Booked + Current PnL: -9.03 L (-6.08%)
-------------------
Total profit:  1.35 L
Total loss:  -21.82 L
-------------------
Total Booked + Current PnL: 17.95 L (14.75%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.22%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 86.34 L (62.06%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.85,58.0,H-MC,3.85,194890.0,30220.0,13116.0,0.24,18.35,6.73,26.31,89.0,2.30,1.40,27.80,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.87,43.0,M-SC,2.89,85274.0,-15503.0,15597.0,0.55,-15.38,18.29,0.09,245.0,-0.99,0.61,12.03,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,56.0,H-MC,10.47,109578.0,9074.0,20031.0,0.32,9.03,18.28,28.96,79.0,0.45,0.79,46.06,MH,ATH,METALS
84,VOLTAS,1530.00,-1.50,48.0,H-MC,3.07,206790.0,15048.0,22706.0,0.46,7.85,10.98,19.69,99.0,0.66,1.49,15.18,XY25,NTT,AC
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.30,214937.0,4571.0,24202.0,-0.38,2.17,11.26,13.68,37.0,0.19,1.55,19.06,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,24.68,64.0,M-LC,2.47,203471.0,-1605.0,114168.0,3.28,-0.78,56.11,54.89,52.0,-0.01,1.46,23.12,X5K,ATH,METALS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82992.0,-19218.0,112612.0,2.23,-18.80,135.69,91.37,208.0,-0.17,0.60,34.72,XY25,NTT,FINANCE
8,AWL,485.00,-65.37,49.0,H-MC,3.27,246390.0,-55361.0,216774.0,2.06,-18.35,87.98,53.50,116.0,-0.26,1.77,9.44,XY24,NTT,FMCG
46,KPIGREEN,731.05,0.19,40.0,H-SC,9.18,119473.0,-5824.0,64754.0,1.91,-4.65,54.20,47.03,141.0,-0.09,0.86,49.59,MH,ATH,POWER
66,SIS,528.00,2083.42,40.0,H-SC,4.44,88115.0,-22917.0,46525.0,1.13,-20.64,52.80,21.26,156.0,-0.49,0.63,19.05,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-42.20,25.0,H-SC,12.75,87530.0,-14970.0,55573.0,-3.55,-14.60,63.49,39.61,114.0,-0.27,0.63,0.00,AR,ATH,MISC
71,TANLA,1963.11,-17.45,62.0,H-SC,16.29,221651.0,-49110.0,379068.0,-3.28,-18.14,171.02,121.86,133.0,-0.13,1.59,73.39,AR,ATH,IT
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.48,94132.0,2634.0,76115.0,-3.16,2.88,80.86,86.07,223.0,0.03,0.68,36.70,XR,NTT,DURABLES
7,ATULAUTO,844.00,3624.32,50.0,M-SC,6.54,116041.0,-31327.0,83143.0,-3.15,-21.26,71.65,35.16,236.0,-0.38,0.83,18.52,XY24,NTT,AUTO
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.89,83155.0,-30394.0,70441.0,-2.91,-26.77,84.71,35.27,268.0,-0.43,0.60,103.52,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.68,41.0,H-MC,2.13,134752.0,-1036.0,43781.0,-0.07,-0.76,32.49,31.48,107.0,-0.02,0.97,11.26,X40,ATH,INSURANCE
23,DABUR,735.00,-7.98,38.0,H-MC,6.83,248926.0,858.0,107561.0,-0.88,0.35,43.21,43.70,102.0,0.01,1.79,14.84,XY24,BTT,FMCG
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,150176.0,-769.0,110229.0,-0.97,-0.51,73.40,72.51,53.0,-0.01,1.08,5.48,XR,NTT,IT
32,HINDZINC,730.22,24.68,64.0,M-LC,2.47,203471.0,-1605.0,114168.0,3.28,-0.78,56.11,54.89,52.0,-0.01,1.46,23.12,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.93,121743.0,-20736.0,40260.0,0.18,-14.55,33.07,13.70,163.0,-0.52,0.88,43.69,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.87,43.0,M-SC,2.89,85274.0,-15503.0,15597.0,0.55,-15.38,18.29,0.09,245.0,-0.99,0.61,12.03,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.27,220779.0,-48888.0,85221.0,0.01,-18.13,38.60,13.47,138.0,-0.57,1.59,11.67,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,34.0,H-SC,1.99,140990.0,-34913.0,76938.0,0.12,-19.85,54.57,23.89,149.0,-0.45,1.01,21.76,OX40N,NTT,CERAMICS
66,SIS,528.0,2083.42,40.0,H-SC,4.44,88115.0,-22917.0,46525.0,1.13,-20.64,52.80,21.26,156.0,-0.49,0.63,19.05,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,70.0,M-MC,10.57,237137.0,12175.0,155491.0,1.01,5.41,65.57,74.53,192.0,0.08,1.71,39.41,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.85,58.0,H-MC,3.85,194890.0,30220.0,13116.0,0.24,18.35,6.73,26.31,89.0,2.30,1.40,27.80,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,48.0,H-MC,3.07,206790.0,15048.0,22706.0,0.46,7.85,10.98,19.69,99.0,0.66,1.49,15.18,XY25,NTT,AC
17,CAMS,5211.76,-3.61,51.0,H-SC,1.81,109623.0,7619.0,36307.0,0.82,7.47,33.12,43.06,122.0,0.21,0.79,25.86,X40N,ATH,MISC
23,DABUR,735.00,-7.98,38.0,H-MC,6.83,248926.0,858.0,107561.0,-0.88,0.35,43.21,43.70,102.0,0.01,1.79,14.84,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,56.0,H-MC,10.47,109578.0,9074.0,20031.0,0.32,9.03,18.28,28.96,79.0,0.45,0.79,46.06,MH,ATH,METALS
11,BANKINDIA,190.00,-27.95,65.0,H-MC,7.90,185714.0,5906.0,106507.0,-0.20,3.28,57.35,62.52,88.0,0.06,1.34,38.49,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.48,94132.0,2634.0,76115.0,-3.16,2.88,80.86,86.07,223.0,0.03,0.68,36.70,XR,NTT,DURABLES
37,INDIAMART,4850.92,-54.64,29.0,H-SC,10.47,126559.0,3223.0,130546.0,0.47,2.61,103.15,108.45,119.0,0.02,0.91,25.55,AR,ATH,MISC
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,150176.0,-769.0,110229.0,-0.97,-0.51,73.40,72.51,53.0,-0.01,1.08,5.48,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-16.20,34.0,H-LC,7.38,211675.0,-40093.0,93306.0,-2.11,-15.92,44.08,21.13,27.0,-0.43,1.52,13.86,X40,ATH,PAINTS
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290226.0,-25616.0,138293.0,0.51,-8.11,47.65,35.67,5.0,-0.19,2.09,4.62,X40N,ATH,FMCG
74,TCS,4476.75,-29.97,34.0,H-LC,12.09,279688.0,-66268.0,141131.0,-1.98,-19.15,50.46,21.64,1.0,-0.47,2.01,0.00,X40,ATH,IT
44,JIOFIN,387.00,-3.34,36.0,H-LC,13.14,262730.0,-4635.0,70096.0,-0.41,-1.73,26.68,24.48,48.0,-0.07,1.89,52.29,XY24,BTT,FINANCE
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194842.0,-5296.0,26187.0,-0.64,-2.65,13.44,10.44,4.0,-0.20,1.40,2.77,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.67,234788.0,-11363.0,90581.0,0.20,-4.62,38.58,32.18,16.0,-0.13,1.69,29.76,X200,ATH,IT
42,ITC,452.0,-39.61,36.0,H-LC,2.11,194842.0,-5296.0,26187.0,-0.64,-2.65,13.44,10.44,4.0,-0.20,1.40,2.77,X40,NTT,FMCG
65,SIEMENS,4671.5,-1.33,47.0,H-LC,2.83,158435.0,-27660.0,75146.0,-1.01,-14.86,47.43,25.51,15.0,-0.37,1.14,16.75,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-10.39,47.0,H-LC,4.23,275582.0,10156.0,46628.0,-0.08,3.83,16.92,21.40,11.0,0.22,1.98,10.72,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,43.0,H-LC,4.30,214937.0,4571.0,24202.0,-0.38,2.17,11.26,13.68,37.0,0.19,1.55,19.06,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.97,34.0,H-LC,12.09,279688.0,-66268.0,141131.0,-1.98,-19.15,50.46,21.64,1.0,-0.47,2.01,0.00,X40,ATH,IT
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194842.0,-5296.0,26187.0,-0.64,-2.65,13.44,10.44,4.0,-0.20,1.40,2.77,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.80,51.0,H-LC,7.12,228674.0,-5470.0,54104.0,-1.58,-2.34,23.66,20.77,86.0,-0.10,1.64,4.42,X40N,NTT,BREWERIES
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290226.0,-25616.0,138293.0,0.51,-8.11,47.65,35.67,5.0,-0.19,2.09,4.62,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.87,223552.0,-18372.0,81641.0,-1.28,-7.59,36.52,26.15,8.0,-0.23,1.61,5.64,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.33,47.0,H-LC,2.83,158435.0,-27660.0,75146.0,-1.01,-14.86,47.43,25.51,15.0,-0.37,1.14,16.75,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194842.0,-5296.0,26187.0,-0.64,-2.65,13.44,10.44,4.0,-0.20,1.40,2.77,X40,NTT,FMCG
19,CIPLA,1795.00,-22.18,40.0,H-LC,4.88,207582.0,3082.0,38340.0,-1.48,1.51,18.47,20.25,10.0,0.08,1.49,10.37,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-16.20,34.0,H-LC,7.38,211675.0,-40093.0,93306.0,-2.11,-15.92,44.08,21.13,27.0,-0.43,1.52,13.86,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.30,214937.0,4571.0,24202.0,-0.38,2.17,11.26,13.68,37.0,0.19,1.55,19.06,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.97,34.0,H-LC,12.09,279688.0,-66268.0,141131.0,-1.98,-19.15,50.46,21.64,1.0,-0.47,2.01,0.00,X40,ATH,IT
40,INFY,2275.00,-18.88,46.0,H-LC,8.09,315986.0,3098.0,168579.0,-0.74,0.99,53.35,54.87,3.0,0.02,2.27,7.71,X40,BTT,IT
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194842.0,-5296.0,26187.0,-0.64,-2.65,13.44,10.44,4.0,-0.20,1.40,2.77,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290226.0,-25616.0,138293.0,0.51,-8.11,47.65,35.67,5.0,-0.19,2.09,4.62,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,8.30,250128.0,-11491.0,130692.0,-0.50,-4.39,52.25,45.57,7.0,-0.09,1.80,6.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6711.11,48.0,L-SC,5.35,74702.0,-19108.0,95723.0,-1.72,-20.37,128.14,81.67,269.0,-0.20,0.54,44.66,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.89,83155.0,-30394.0,70441.0,-2.91,-26.77,84.71,35.27,268.0,-0.43,0.60,103.52,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82992.0,-19218.0,112612.0,2.23,-18.80,135.69,91.37,208.0,-0.17,0.60,34.72,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-7.20,39.0,H-SC,11.43,86344.0,-14619.0,109147.0,-0.77,-14.48,126.41,93.63,148.0,-0.13,0.62,28.61,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,49.0,H-SC,12.67,93840.0,-4140.0,25740.0,-1.06,-4.23,27.43,22.05,152.0,-0.16,0.67,36.40,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1195.06,68.0,M-SC,11.42,176126.0,-5017.0,50478.0,-1.71,-2.77,28.66,25.10,235.0,-0.10,1.27,41.15,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-17.45,62.0,H-SC,16.29,221651.0,-49110.0,379068.0,-3.28,-18.14,171.02,121.86,133.0,-0.13,1.59,73.39,AR,ATH,IT
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82992.0,-19218.0,112612.0,2.23,-18.80,135.69,91.37,208.0,-0.17,0.60,34.72,XY25,NTT,FINANCE
60,SAIL,228.00,49.96,70.0,M-MC,10.57,237137.0,12175.0,155491.0,1.01,5.41,65.57,74.53,192.0,0.08,1.71,39.41,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,58.13,57.0,H-SC,5.95,137193.0,-45582.0,156647.0,-0.84,-24.94,114.18,60.77,125.0,-0.29,0.99,22.88,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,43.71
2,50,74.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.39
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.30
X40      14.73
X40N     11.41
XR       11.23
XY25     10.15
AR        8.32
OX40N     7.86
X200      1.69
MH        1.65
X5K       1.46
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.91
H-MC    20.74
M-SC    13.03
M-LC     5.43
M-MC     2.68
L-SC     1.50
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.00,-5.45,40.26
IT,12.90,-17.10,80.95
FINANCE,10.07,-17.18,68.31
MISC,6.99,-18.16,83.85
BANKS,6.20,-16.05,77.09
PAINTS,5.64,-17.65,35.18
ELECTRICAL,5.47,-8.18,46.13
INSURANCE,3.76,-5.82,45.38
AC,3.59,2.97,13.27


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3167658.0,22
XR,1349162.0,14
AR,1174576.0,9
X40,800101.0,10
X40N,619394.0,9
OX40N,552443.0,10
XY25,429803.0,7
SR,251391.0,2
X5K,114168.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3706053.0,29
M-SC,1435648.0,17
H-MC,1223801.0,15
H-LC,1216730.0,15
M-LC,394831.0,4
M-MC,305401.0,2
L-SC,252974.0,3
L-MC,60303.0,1
L-LC,38321.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237331.0      6
M-SC       XY24       821771.0      7
H-SC       AR         799684.0      5
           XR         783309.0      7
H-MC       XY24       562625.0      4
H-LC       X40        510844.0      5
H-SC       X40N       309391.0      4
M-MC       XY24       305401.0      2
H-SC       OX40N      260193.0      4
           SR         251391.0      2
H-LC       X40N       230737.0      3
H-MC       X40        215143.0      4
H-LC       AR         205838.0      2
M-LC       XY24       170434.0      2
H-MC       XY25       170236.0      2
M-SC       AR         169054.0      2
L-SC       XR         166164.0      2
M-SC       OX40N      135447.0      4
           XR         122650.0      2
M-LC       X5K        114168.0      1
M-SC       XY25       112612.0      1
M-LC       XR         110229.0      1
H-LC       XY25       108634.0      3
H-MC       XR         106507.0      1
H-LC       X200        90581.0      1
L-SC       OX40N       86810.0      1
H-MC       X40N        79266.0      2
M-SC       X40         74114.0      1
H-LC       XY24        70096.0      1
H-MC       OX40N       69993.0      1
H-SC       MH          64754.0      1
L-MC       XR          60303.0      1
L-LC       XY25        38321.0      1
H-MC       MH          20031.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
